In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.version

'3.8.16 | packaged by conda-forge | (default, Feb  1 2023, 15:53:35) [MSC v.1929 64 bit (AMD64)]'

In [2]:
# import tensorflow as tf

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, QuantileTransformer

import cv2
from PIL import Image

import math
import random

import os
import shutil
import joblib

# 设置显示的最大行数和列数
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
import seaborn as sns

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support
from sklearn.metrics import f1_score,roc_auc_score

In [37]:
df = pd.read_csv('data/DNN-EdgeIIoT-dataset.csv', low_memory=False)

print(df.shape)
df.head()

(2219201, 63)


,frame.time,ip.src_host,ip.dst_host,arp.dst.proto_ipv4,arp.opcode,arp.hw.size,arp.src.proto_ipv4,icmp.checksum,icmp.seq_le,icmp.transmit_timestamp,icmp.unused,http.file_data,http.content_length,http.request.uri.query,http.request.method,http.referer,http.request.full_uri,http.request.version,http.response,http.tls_port,tcp.ack,tcp.ack_raw,tcp.checksum,tcp.connection.fin,tcp.connection.rst,tcp.connection.syn,tcp.connection.synack,tcp.dstport,tcp.flags,tcp.flags.ack,tcp.len,tcp.options,tcp.payload,tcp.seq,tcp.srcport,udp.port,udp.stream,udp.time_delta,dns.qry.name,dns.qry.name.len,dns.qry.qu,dns.qry.type,dns.retransmission,dns.retransmit_request,dns.retransmit_request_in,mqtt.conack.flags,mqtt.conflag.cleansess,mqtt.conflags,mqtt.hdrflags,mqtt.len,mqtt.msg_decoded_as,mqtt.msg,mqtt.msgtype,mqtt.proto_len,mqtt.protoname,mqtt.topic,mqtt.topic_len,mqtt.ver,mbtcp.len,mbtcp.trans_id,mbtcp.unit_id,Attack_label,Attack_type
0,2021 11:44:10.081753000,192.168.0.128,192.168.0.101,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.153346e+09,47892.0,0.0,0.0,0.0,1.0,64855.0,18.0,1.0,0.0,020405b40101040201030307,0,0.0,1883.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,Normal
1,2021 11:44:10.162218000,192.168.0.101,192.168.0.128,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.447945e+08,16077.0,0.0,0.0,0.0,0.0,1883.0,24.0,1.0,14.0,0,100c00044d5154540402003c0000,1.0,64855.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,1.0,2.0,16.0,12.0,0.0,0,1.0,4.0,MQTT,0,0.0,4.0,0.0,0.0,0.0,0,Normal
2,2021 11:44:10.162271000,192.168.0.128,192.168.0.101,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,3.153346e+09,62675.0,0.0,0.0,0.0,0.0,64855.0,16.0,1.0,0.0,0,0,1.0,1883.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,Normal
3,2021 11:44:10.162641000,192.168.0.128,192.168.0.101,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,3.153346e+09,54469.0,0.0,0.0,0.0,0.0,64855.0,24.0,1.0,4.0,0,20020000,1.0,1883.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0x00000000,0.0,0.0,32.0,2.0,0.0,0,2.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,Normal
4,2021 11:44:10.166132000,192.168.0.101,192.168.0.128,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,3.447945e+08,53812.0,0.0,0.0,0.0,0.0,1883.0,24.0,1.0,41.0,0,3027001854656d70657261747572655f616e645f48756d...,15.0,64855.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,48.0,39.0,0.0,32342e36382037362e34320d0a,3.0,0.0,0,Temperature_and_Humidity,24.0,0.0,0.0,0.0,0.0,0,Normal


### Delete column

In [38]:
print('Data column BEFORE deleteting numbers: ', df.shape[1], end='\n\n')

drop_columns = ["frame.time", "ip.src_host", "ip.dst_host", "arp.src.proto_ipv4","arp.dst.proto_ipv4", 
                "http.file_data","http.request.full_uri","icmp.transmit_timestamp", 
                "http.request.uri.query", "tcp.options","tcp.payload","tcp.srcport", 
                "tcp.dstport", "udp.port", "mqtt.msg"]

df.drop(drop_columns, axis=1, inplace=True)

print('Data column AFTER deleteting numbers: ', df.shape[1])

Data column BEFORE deleteting numbers:  63

Data column AFTER deleteting numbers:  48


### Dealing with duplicates

In [39]:
df.duplicated().any()

True

In [40]:
print('Data size BEFORE deleteting instances with duplicate values: ', df.shape[0], end='\n\n')

df.drop_duplicates(inplace=True, keep=False, ignore_index=True)

print('Data size AFTER deleteting instances containing duplicate values: ', df.shape[0])

Data size BEFORE deleteting instances with duplicate values:  2219201

Data size AFTER deleteting instances containing duplicate values:  1720188


### Dealing with NULL

In [41]:
df.columns[df.isnull().any()]

Index([], dtype='object')

In [42]:
# 计算每个特征的缺失值数量
missing_values_count = df.isnull().sum()
print(missing_values_count)

arp.opcode                   0
arp.hw.size                  0
icmp.checksum                0
icmp.seq_le                  0
icmp.unused                  0
http.content_length          0
http.request.method          0
http.referer                 0
http.request.version         0
http.response                0
http.tls_port                0
tcp.ack                      0
tcp.ack_raw                  0
tcp.checksum                 0
tcp.connection.fin           0
tcp.connection.rst           0
tcp.connection.syn           0
tcp.connection.synack        0
tcp.flags                    0
tcp.flags.ack                0
tcp.len                      0
tcp.seq                      0
udp.stream                   0
udp.time_delta               0
dns.qry.name                 0
dns.qry.name.len             0
dns.qry.qu                   0
dns.qry.type                 0
dns.retransmission           0
dns.retransmit_request       0
dns.retransmit_request_in    0
mqtt.conack.flags            0
mqtt.con

In [43]:
# # 获取特征名称和列号的映射关系
# column_name_to_index = {name: index for index, name in enumerate(df.columns)}
# missing_values_features = missing_values_count[missing_values_count > 0]

# # 输出包含缺失值的特征名称、列号及其缺失值数量
# print("包含缺失值的特征名称、列号及其缺失值数量：")
# for feature, count in missing_values_features.items():
#     print(f"{feature} (列号: {column_name_to_index[feature]}) - 缺失值数量: {count}")

In [44]:
# # Remove missing values

# print('Data size BEFORE deleteting instances with missing values: ', df.shape[0], end='\n\n')

# df.dropna(axis=0, inplace=True, how="any")

# print('Data size AFTER deleteting instances containing missing values: ', df.shape[0])

### 数据格式处理

In [45]:
print(df.dtypes)

arp.opcode                   float64
arp.hw.size                  float64
icmp.checksum                float64
icmp.seq_le                  float64
icmp.unused                  float64
http.content_length          float64
http.request.method           object
http.referer                  object
http.request.version          object
http.response                float64
http.tls_port                float64
tcp.ack                      float64
tcp.ack_raw                  float64
tcp.checksum                 float64
tcp.connection.fin           float64
tcp.connection.rst           float64
tcp.connection.syn           float64
tcp.connection.synack        float64
tcp.flags                    float64
tcp.flags.ack                float64
tcp.len                      float64
tcp.seq                      float64
udp.stream                   float64
udp.time_delta               float64
dns.qry.name                 float64
dns.qry.name.len              object
dns.qry.qu                   float64
d

In [46]:
labelencoder = LabelEncoder()

un_features = list(df.dtypes[df.dtypes == 'object'].index)
un_features.remove('Attack_type')


for cl in un_features:
    
    print(cl)
    
    # 数据替换
    df[cl] = df[cl].replace('0', '0.0')
    df.loc[df[cl].str.contains('HTTP/1.1'), cl] = 'HTTP/1.1'
    df.loc[df[cl].str.contains('debian.pool.ntp.org'), cl] = 'debian.pool.ntp.org'
    df.loc[df[cl].str.contains('.local'), cl] = 'local'
    
    print(df[cl].value_counts(), end='\n\n\n')
    df[cl] = labelencoder.fit_transform(df[cl])
    
#     del df[cl]


try:
    print(df['Attack_label'].value_counts())
    del df['Attack_label']
except:
    pass


df.head()

http.request.method
0.0         1688103
GET           30472
POST           1212
TRACE           387
OPTIONS           6
PROPFIND          4
PUT               2
SEARCH            2
Name: http.request.method, dtype: int64


http.referer
0.0                                                                     1719618
127.0.0.1                                                                   383
() { _; } >_[$($())] { echo 93e4r0-CVE-2014-6278: true; echo;echo; }        185
TESTING_PURPOSES_ONLY                                                         2
Name: http.referer, dtype: int64


http.request.version
0.0         1688103
HTTP/1.1      28703
HTTP/1.0       3382
Name: http.request.version, dtype: int64


dns.qry.name.len
0.0                    1718136
debian.pool.ntp.org       1775
1.0                        227
local                       50
Name: dns.qry.name.len, dtype: int64


mqtt.conack.flags
0.0           1637131
0x00000000      83017
1574358             9
1574359             9


,arp.opcode,arp.hw.size,icmp.checksum,icmp.seq_le,icmp.unused,http.content_length,http.request.method,http.referer,http.request.version,http.response,http.tls_port,tcp.ack,tcp.ack_raw,tcp.checksum,tcp.connection.fin,tcp.connection.rst,tcp.connection.syn,tcp.connection.synack,tcp.flags,tcp.flags.ack,tcp.len,tcp.seq,udp.stream,udp.time_delta,dns.qry.name,dns.qry.name.len,dns.qry.qu,dns.qry.type,dns.retransmission,dns.retransmit_request,dns.retransmit_request_in,mqtt.conack.flags,mqtt.conflag.cleansess,mqtt.conflags,mqtt.hdrflags,mqtt.len,mqtt.msg_decoded_as,mqtt.msgtype,mqtt.proto_len,mqtt.protoname,mqtt.topic,mqtt.topic_len,mqtt.ver,mbtcp.len,mbtcp.trans_id,mbtcp.unit_id,Attack_type
0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0.0,0.0,1.0,3.153346e+09,47892.0,0.0,0.0,0.0,1.0,18.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,Normal
1,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0.0,0.0,1.0,3.447945e+08,16077.0,0.0,0.0,0.0,0.0,24.0,1.0,14.0,1.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,1.0,2.0,16.0,12.0,0.0,1.0,4.0,1,0,0.0,4.0,0.0,0.0,0.0,Normal
2,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0.0,0.0,15.0,3.153346e+09,62675.0,0.0,0.0,0.0,0.0,16.0,1.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,Normal
3,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0.0,0.0,15.0,3.153346e+09,54469.0,0.0,0.0,0.0,0.0,24.0,1.0,4.0,1.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,32.0,2.0,0.0,2.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,Normal
4,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0.0,0.0,5.0,3.447945e+08,53812.0,0.0,0.0,0.0,0.0,24.0,1.0,41.0,15.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,48.0,39.0,0.0,3.0,0.0,0,1,24.0,0.0,0.0,0.0,0.0,Normal


In [47]:
print(list(df.dtypes[df.dtypes == 'object'].index))

print(df['Attack_type'].value_counts())

['Attack_type']
Normal                   1208176
DDoS_UDP                  121566
SQL_injection              50506
DDoS_TCP                   50062
Vulnerability_scanner      49998
Password                   49785
DDoS_HTTP                  47267
DDoS_ICMP                  36981
Uploading                  36056
Backdoor                   24016
Port_Scanning              19974
XSS                        14949
Ransomware                  9682
Fingerprinting               849
MITM                         321
Name: Attack_type, dtype: int64


### Dealing with infinite values

In [48]:
# Checking if all values are finite
np.all(np.isfinite(df.drop(['Attack_type'], axis=1)))

True

In [49]:
# Replace infinite values to NaN
df.replace([-np.inf, np.inf], np.nan, inplace=True)

In [50]:
# Remove infinte values

print('Data size BEFORE deleteting instances with infinite values: ', df.shape[0], end='\n\n')

df.dropna(axis=0, how='any', inplace=True)

print('Data size AFTER deleteting instances containing infinite values: ', df.shape[0])

Data size BEFORE deleteting instances with infinite values:  1720188

Data size AFTER deleteting instances containing infinite values:  1720188


### Dealing with features with quasi null std deviation

In [51]:
# 计算数据集中所有数值型特征的标准差（std）
# numeric_only=True参数指定只计算数值型特征的标准差，排除了字符串等非数值型特征

df_std = df.std(numeric_only=True)
df_std

arp.opcode                   1.704892e-03
arp.hw.size                  6.469611e-03
icmp.checksum                5.542092e+03
icmp.seq_le                  6.265450e+03
icmp.unused                  0.000000e+00
http.content_length          1.094794e+02
http.request.method          1.863923e-01
http.referer                 1.829846e-02
http.request.version         2.597329e-01
http.response                1.363596e-01
http.tls_port                0.000000e+00
tcp.ack                      1.867763e+08
tcp.ack_raw                  1.265266e+09
tcp.checksum                 2.039313e+04
tcp.connection.fin           3.153815e-01
tcp.connection.rst           1.548249e-01
tcp.connection.syn           2.243893e-01
tcp.connection.synack        2.287194e-01
tcp.flags                    7.429967e+00
tcp.flags.ack                3.566624e-01
tcp.len                      4.216547e+02
tcp.seq                      3.811763e+07
udp.stream                   4.127653e+05
udp.time_delta               1.290

In [52]:
print('Data columns BEFORE deleteting instances with std deviation: ', df.shape[1], end='\n\n')

# Find Features that meet the threshold
constant_features = [column for column, std in df_std.items() if std < 0.01]

# Drop the constant features
df.drop(labels=constant_features, axis=1, inplace=True)

print('Data columns AFTER deleteting instances containing std deviation: ', df.shape[1])

Data columns BEFORE deleteting instances with std deviation:  47

Data columns AFTER deleteting instances containing std deviation:  39


### Preprocessing (normalization and padding values)

In [53]:
# Z-score normalization
features = df.dtypes[df.dtypes != 'object'].index

df[features] = df[features].apply(
    lambda x: (x - x.mean()) / (x.std()))

In [21]:
df.to_csv('data/DLL_clean.csv',index=0)

### Data sampling

In [54]:
def typicalSampling(group):
    name = group.name
    frac = 0.2
    return group.sample(frac=frac)

In [55]:
print(df['Attack_type'].value_counts())

Normal                   1208176
DDoS_UDP                  121566
SQL_injection              50506
DDoS_TCP                   50062
Vulnerability_scanner      49998
Password                   49785
DDoS_HTTP                  47267
DDoS_ICMP                  36981
Uploading                  36056
Backdoor                   24016
Port_Scanning              19974
XSS                        14949
Ransomware                  9682
Fingerprinting               849
MITM                         321
Name: Attack_type, dtype: int64


In [56]:
df_normal = df[(df['Attack_type']=='Normal')]
df_else = df.drop(df_normal.index)

df_minor = df_else[(df_else['Attack_type']=='MITM')|(df_else['Attack_type']=='Fingerprinting')]
df_major = df_else.drop(df_minor.index)

# 随机选择数据
# df_normal = df_normal.sample(frac=0.42)
df_normal = df_normal.sample(n=df_major.shape[0], random_state=2023)

df_major = pd.concat([df_major, df_normal])
del df_normal

In [57]:
X = df_major.drop(['Attack_type'],axis=1) 
y = df_major.iloc[:, -1].values.reshape(-1,1)
y = np.ravel(y)

In [58]:
from sklearn.cluster import MiniBatchKMeans

# use k-means to cluster the data samples and select a proportion of data from each cluster
kmeans = MiniBatchKMeans(n_clusters=1000, random_state=0).fit(X)

klabel=kmeans.labels_
df_major['klabel']=klabel

# df_major['klabel'].value_counts()

In [59]:
cols = list(df_major)
cols.insert(df.shape[1], cols.pop(cols.index('Attack_type')))
df_major = df_major.loc[:, cols]

In [60]:
result = df_major.groupby(
    'klabel', group_keys=False
).apply(typicalSampling)


print(result.shape)
result.head()

(204345, 40)


,icmp.checksum,icmp.seq_le,http.content_length,http.request.method,http.referer,http.request.version,http.response,tcp.ack,tcp.ack_raw,tcp.checksum,tcp.connection.fin,tcp.connection.rst,tcp.connection.syn,tcp.connection.synack,tcp.flags,tcp.flags.ack,tcp.len,tcp.seq,udp.stream,udp.time_delta,dns.qry.name,dns.qry.name.len,dns.qry.qu,dns.retransmission,mqtt.conack.flags,mqtt.conflag.cleansess,mqtt.conflags,mqtt.hdrflags,mqtt.len,mqtt.msgtype,mqtt.proto_len,mqtt.protoname,mqtt.topic,mqtt.topic_len,mqtt.ver,mbtcp.len,mbtcp.trans_id,mbtcp.unit_id,klabel,Attack_type
917290,-0.129126,-0.147243,-0.05666,-0.114983,-0.006417,-0.136055,-0.138994,-0.157324,-1.295406,-0.285951,-0.355164,-0.158725,-0.236992,4.130041,0.212608,0.419397,-0.359221,-0.282489,-0.224993,-0.034148,-0.07032,-0.033972,-0.013281,-0.023533,-0.222261,-0.225163,-0.225163,-0.318158,-0.297317,-0.318158,-0.225163,-0.225163,-0.225179,-0.225179,-0.225163,-0.008613,-0.008127,-0.007767,0,Normal
1219577,-0.129126,-0.147243,-0.05666,-0.114983,-0.006417,-0.136055,-0.138994,-0.157324,-1.101759,-0.382258,-0.355164,-0.158725,-0.236992,4.130041,0.212608,0.419397,-0.359221,-0.282489,-0.224993,-0.034148,-0.07032,-0.033972,-0.013281,-0.023533,-0.222261,-0.225163,-0.225163,-0.318158,-0.297317,-0.318158,-0.225163,-0.225163,-0.225179,-0.225179,-0.225163,-0.008613,-0.008127,-0.007767,0,Uploading
1097787,-0.129126,-0.147243,-0.05666,-0.114983,-0.006417,-0.136055,-0.138994,-0.157324,-1.350926,-0.304977,-0.355164,-0.158725,-0.236992,4.130041,0.212608,0.419397,-0.359221,-0.282489,-0.224993,-0.034148,-0.07032,-0.033972,-0.013281,-0.023533,-0.222261,-0.225163,-0.225163,-0.318158,-0.297317,-0.318158,-0.225163,-0.225163,-0.225179,-0.225179,-0.225163,-0.008613,-0.008127,-0.007767,0,Normal
44793,-0.129126,-0.147243,-0.05666,-0.114983,-0.006417,-0.136055,-0.138994,-0.157324,-1.298922,-0.389319,-0.355164,-0.158725,-0.236992,4.130041,0.212608,0.419397,-0.359221,-0.282489,-0.224993,-0.034148,-0.07032,-0.033972,-0.013281,-0.023533,-0.222261,-0.225163,-0.225163,-0.318158,-0.297317,-0.318158,-0.225163,-0.225163,-0.225179,-0.225179,-0.225163,-0.008613,-0.008127,-0.007767,0,Normal
1284011,-0.129126,-0.147243,-0.05666,-0.114983,-0.006417,-0.136055,-0.138994,-0.157324,-1.084731,-0.415750,-0.355164,-0.158725,-0.236992,4.130041,0.212608,0.419397,-0.359221,-0.282489,-0.224993,-0.034148,-0.07032,-0.033972,-0.013281,-0.023533,-0.222261,-0.225163,-0.225163,-0.318158,-0.297317,-0.318158,-0.225163,-0.225163,-0.225179,-0.225179,-0.225163,-0.008613,-0.008127,-0.007767,0,SQL_injection


In [61]:
print(result['Attack_type'].value_counts())

Normal                   102102
DDoS_UDP                  24339
SQL_injection             10008
Password                   9986
DDoS_TCP                   9958
Vulnerability_scanner      9950
DDoS_HTTP                  9509
DDoS_ICMP                  7370
Uploading                  7306
Backdoor                   4832
Port_Scanning              4033
XSS                        3046
Ransomware                 1906
Name: Attack_type, dtype: int64


In [62]:
print(df_minor['Attack_type'].value_counts())

Fingerprinting    849
MITM              321
Name: Attack_type, dtype: int64


In [63]:
result = result.drop(['klabel'],axis=1)
result = pd.concat([result, df_minor])

print(result.shape)
result.head()

(205515, 39)


,icmp.checksum,icmp.seq_le,http.content_length,http.request.method,http.referer,http.request.version,http.response,tcp.ack,tcp.ack_raw,tcp.checksum,tcp.connection.fin,tcp.connection.rst,tcp.connection.syn,tcp.connection.synack,tcp.flags,tcp.flags.ack,tcp.len,tcp.seq,udp.stream,udp.time_delta,dns.qry.name,dns.qry.name.len,dns.qry.qu,dns.retransmission,mqtt.conack.flags,mqtt.conflag.cleansess,mqtt.conflags,mqtt.hdrflags,mqtt.len,mqtt.msgtype,mqtt.proto_len,mqtt.protoname,mqtt.topic,mqtt.topic_len,mqtt.ver,mbtcp.len,mbtcp.trans_id,mbtcp.unit_id,Attack_type
917290,-0.129126,-0.147243,-0.05666,-0.114983,-0.006417,-0.136055,-0.138994,-0.157324,-1.295406,-0.285951,-0.355164,-0.158725,-0.236992,4.130041,0.212608,0.419397,-0.359221,-0.282489,-0.224993,-0.034148,-0.07032,-0.033972,-0.013281,-0.023533,-0.222261,-0.225163,-0.225163,-0.318158,-0.297317,-0.318158,-0.225163,-0.225163,-0.225179,-0.225179,-0.225163,-0.008613,-0.008127,-0.007767,Normal
1219577,-0.129126,-0.147243,-0.05666,-0.114983,-0.006417,-0.136055,-0.138994,-0.157324,-1.101759,-0.382258,-0.355164,-0.158725,-0.236992,4.130041,0.212608,0.419397,-0.359221,-0.282489,-0.224993,-0.034148,-0.07032,-0.033972,-0.013281,-0.023533,-0.222261,-0.225163,-0.225163,-0.318158,-0.297317,-0.318158,-0.225163,-0.225163,-0.225179,-0.225179,-0.225163,-0.008613,-0.008127,-0.007767,Uploading
1097787,-0.129126,-0.147243,-0.05666,-0.114983,-0.006417,-0.136055,-0.138994,-0.157324,-1.350926,-0.304977,-0.355164,-0.158725,-0.236992,4.130041,0.212608,0.419397,-0.359221,-0.282489,-0.224993,-0.034148,-0.07032,-0.033972,-0.013281,-0.023533,-0.222261,-0.225163,-0.225163,-0.318158,-0.297317,-0.318158,-0.225163,-0.225163,-0.225179,-0.225179,-0.225163,-0.008613,-0.008127,-0.007767,Normal
44793,-0.129126,-0.147243,-0.05666,-0.114983,-0.006417,-0.136055,-0.138994,-0.157324,-1.298922,-0.389319,-0.355164,-0.158725,-0.236992,4.130041,0.212608,0.419397,-0.359221,-0.282489,-0.224993,-0.034148,-0.07032,-0.033972,-0.013281,-0.023533,-0.222261,-0.225163,-0.225163,-0.318158,-0.297317,-0.318158,-0.225163,-0.225163,-0.225179,-0.225179,-0.225163,-0.008613,-0.008127,-0.007767,Normal
1284011,-0.129126,-0.147243,-0.05666,-0.114983,-0.006417,-0.136055,-0.138994,-0.157324,-1.084731,-0.415750,-0.355164,-0.158725,-0.236992,4.130041,0.212608,0.419397,-0.359221,-0.282489,-0.224993,-0.034148,-0.07032,-0.033972,-0.013281,-0.023533,-0.222261,-0.225163,-0.225163,-0.318158,-0.297317,-0.318158,-0.225163,-0.225163,-0.225179,-0.225179,-0.225163,-0.008613,-0.008127,-0.007767,SQL_injection


In [64]:
print(result['Attack_type'].value_counts())

Normal                   102102
DDoS_UDP                  24339
SQL_injection             10008
Password                   9986
DDoS_TCP                   9958
Vulnerability_scanner      9950
DDoS_HTTP                  9509
DDoS_ICMP                  7370
Uploading                  7306
Backdoor                   4832
Port_Scanning              4033
XSS                        3046
Ransomware                 1906
Fingerprinting              849
MITM                        321
Name: Attack_type, dtype: int64


In [65]:
result.to_csv('data/DLL_km.csv',index=0)

In [68]:
# Read the sampled dataset
df = pd.read_csv('data/DLL_km.csv', low_memory=False)

print(df.shape)
df.head()

(205515, 39)


,icmp.checksum,icmp.seq_le,http.content_length,http.request.method,http.referer,http.request.version,http.response,tcp.ack,tcp.ack_raw,tcp.checksum,tcp.connection.fin,tcp.connection.rst,tcp.connection.syn,tcp.connection.synack,tcp.flags,tcp.flags.ack,tcp.len,tcp.seq,udp.stream,udp.time_delta,dns.qry.name,dns.qry.name.len,dns.qry.qu,dns.retransmission,mqtt.conack.flags,mqtt.conflag.cleansess,mqtt.conflags,mqtt.hdrflags,mqtt.len,mqtt.msgtype,mqtt.proto_len,mqtt.protoname,mqtt.topic,mqtt.topic_len,mqtt.ver,mbtcp.len,mbtcp.trans_id,mbtcp.unit_id,Attack_type
0,-0.129126,-0.147243,-0.05666,-0.114983,-0.006417,-0.136055,-0.138994,-0.157324,-1.295406,-0.285951,-0.355164,-0.158725,-0.236992,4.130041,0.212608,0.419397,-0.359221,-0.282489,-0.224993,-0.034148,-0.07032,-0.033972,-0.013281,-0.023533,-0.222261,-0.225163,-0.225163,-0.318158,-0.297317,-0.318158,-0.225163,-0.225163,-0.225179,-0.225179,-0.225163,-0.008613,-0.008127,-0.007767,Normal
1,-0.129126,-0.147243,-0.05666,-0.114983,-0.006417,-0.136055,-0.138994,-0.157324,-1.101759,-0.382258,-0.355164,-0.158725,-0.236992,4.130041,0.212608,0.419397,-0.359221,-0.282489,-0.224993,-0.034148,-0.07032,-0.033972,-0.013281,-0.023533,-0.222261,-0.225163,-0.225163,-0.318158,-0.297317,-0.318158,-0.225163,-0.225163,-0.225179,-0.225179,-0.225163,-0.008613,-0.008127,-0.007767,Uploading
2,-0.129126,-0.147243,-0.05666,-0.114983,-0.006417,-0.136055,-0.138994,-0.157324,-1.350926,-0.304977,-0.355164,-0.158725,-0.236992,4.130041,0.212608,0.419397,-0.359221,-0.282489,-0.224993,-0.034148,-0.07032,-0.033972,-0.013281,-0.023533,-0.222261,-0.225163,-0.225163,-0.318158,-0.297317,-0.318158,-0.225163,-0.225163,-0.225179,-0.225179,-0.225163,-0.008613,-0.008127,-0.007767,Normal
3,-0.129126,-0.147243,-0.05666,-0.114983,-0.006417,-0.136055,-0.138994,-0.157324,-1.298922,-0.389319,-0.355164,-0.158725,-0.236992,4.130041,0.212608,0.419397,-0.359221,-0.282489,-0.224993,-0.034148,-0.07032,-0.033972,-0.013281,-0.023533,-0.222261,-0.225163,-0.225163,-0.318158,-0.297317,-0.318158,-0.225163,-0.225163,-0.225179,-0.225179,-0.225163,-0.008613,-0.008127,-0.007767,Normal
4,-0.129126,-0.147243,-0.05666,-0.114983,-0.006417,-0.136055,-0.138994,-0.157324,-1.084731,-0.415750,-0.355164,-0.158725,-0.236992,4.130041,0.212608,0.419397,-0.359221,-0.282489,-0.224993,-0.034148,-0.07032,-0.033972,-0.013281,-0.023533,-0.222261,-0.225163,-0.225163,-0.318158,-0.297317,-0.318158,-0.225163,-0.225163,-0.225179,-0.225179,-0.225163,-0.008613,-0.008127,-0.007767,SQL_injection


In [69]:
df['Attack_type'].value_counts()

Normal                   102102
DDoS_UDP                  24339
SQL_injection             10008
Password                   9986
DDoS_TCP                   9958
Vulnerability_scanner      9950
DDoS_HTTP                  9509
DDoS_ICMP                  7370
Uploading                  7306
Backdoor                   4832
Port_Scanning              4033
XSS                        3046
Ransomware                 1906
Fingerprinting              849
MITM                        321
Name: Attack_type, dtype: int64

In [70]:
# 定义目标采样策略
sampling_strategy = {'Fingerprinting': 15000, 'MITM': 15000, 'Ransomware': 15000, 'XSS':15000, 
                     'Port_Scanning':15000, 'Backdoor':15000, 'Uploading':15000, 'DDoS_ICMP':15000, 
                     'DDoS_HTTP':15000, 'Password':15000, 'DDoS_TCP':15000, 'Vulnerability_scanner':15000, 
                     'SQL_injection':15000}

# 创建SMOTE对象
smote = SMOTE(sampling_strategy=sampling_strategy, n_jobs=-1)

# 进行过采样
X_resampled, y_resampled = smote.fit_resample(df.drop('Attack_type', axis=1), df['Attack_type'])

In [71]:
# 将特征和目标变量转换为DataFrame
X_resampled_df = pd.DataFrame(X_resampled, columns=df.drop('Attack_type', axis=1).columns)
y_resampled_df = pd.DataFrame(y_resampled, columns=['Attack_type'])

df = pd.concat([X_resampled_df, y_resampled_df], axis=1)

print(df.shape)
df.head()

(321441, 39)


,icmp.checksum,icmp.seq_le,http.content_length,http.request.method,http.referer,http.request.version,http.response,tcp.ack,tcp.ack_raw,tcp.checksum,tcp.connection.fin,tcp.connection.rst,tcp.connection.syn,tcp.connection.synack,tcp.flags,tcp.flags.ack,tcp.len,tcp.seq,udp.stream,udp.time_delta,dns.qry.name,dns.qry.name.len,dns.qry.qu,dns.retransmission,mqtt.conack.flags,mqtt.conflag.cleansess,mqtt.conflags,mqtt.hdrflags,mqtt.len,mqtt.msgtype,mqtt.proto_len,mqtt.protoname,mqtt.topic,mqtt.topic_len,mqtt.ver,mbtcp.len,mbtcp.trans_id,mbtcp.unit_id,Attack_type
0,-0.129126,-0.147243,-0.05666,-0.114983,-0.006417,-0.136055,-0.138994,-0.157324,-1.295406,-0.285951,-0.355164,-0.158725,-0.236992,4.130041,0.212608,0.419397,-0.359221,-0.282489,-0.224993,-0.034148,-0.07032,-0.033972,-0.013281,-0.023533,-0.222261,-0.225163,-0.225163,-0.318158,-0.297317,-0.318158,-0.225163,-0.225163,-0.225179,-0.225179,-0.225163,-0.008613,-0.008127,-0.007767,Normal
1,-0.129126,-0.147243,-0.05666,-0.114983,-0.006417,-0.136055,-0.138994,-0.157324,-1.101759,-0.382258,-0.355164,-0.158725,-0.236992,4.130041,0.212608,0.419397,-0.359221,-0.282489,-0.224993,-0.034148,-0.07032,-0.033972,-0.013281,-0.023533,-0.222261,-0.225163,-0.225163,-0.318158,-0.297317,-0.318158,-0.225163,-0.225163,-0.225179,-0.225179,-0.225163,-0.008613,-0.008127,-0.007767,Uploading
2,-0.129126,-0.147243,-0.05666,-0.114983,-0.006417,-0.136055,-0.138994,-0.157324,-1.350926,-0.304977,-0.355164,-0.158725,-0.236992,4.130041,0.212608,0.419397,-0.359221,-0.282489,-0.224993,-0.034148,-0.07032,-0.033972,-0.013281,-0.023533,-0.222261,-0.225163,-0.225163,-0.318158,-0.297317,-0.318158,-0.225163,-0.225163,-0.225179,-0.225179,-0.225163,-0.008613,-0.008127,-0.007767,Normal
3,-0.129126,-0.147243,-0.05666,-0.114983,-0.006417,-0.136055,-0.138994,-0.157324,-1.298922,-0.389319,-0.355164,-0.158725,-0.236992,4.130041,0.212608,0.419397,-0.359221,-0.282489,-0.224993,-0.034148,-0.07032,-0.033972,-0.013281,-0.023533,-0.222261,-0.225163,-0.225163,-0.318158,-0.297317,-0.318158,-0.225163,-0.225163,-0.225179,-0.225179,-0.225163,-0.008613,-0.008127,-0.007767,Normal
4,-0.129126,-0.147243,-0.05666,-0.114983,-0.006417,-0.136055,-0.138994,-0.157324,-1.084731,-0.415750,-0.355164,-0.158725,-0.236992,4.130041,0.212608,0.419397,-0.359221,-0.282489,-0.224993,-0.034148,-0.07032,-0.033972,-0.013281,-0.023533,-0.222261,-0.225163,-0.225163,-0.318158,-0.297317,-0.318158,-0.225163,-0.225163,-0.225179,-0.225179,-0.225163,-0.008613,-0.008127,-0.007767,SQL_injection


In [72]:
df['Attack_type'].value_counts()

Normal                   102102
DDoS_UDP                  24339
Uploading                 15000
SQL_injection             15000
Vulnerability_scanner     15000
Password                  15000
XSS                       15000
DDoS_HTTP                 15000
Port_Scanning             15000
Backdoor                  15000
Ransomware                15000
DDoS_TCP                  15000
DDoS_ICMP                 15000
MITM                      15000
Fingerprinting            15000
Name: Attack_type, dtype: int64

### 分类转换

DDoS攻击类别：

DDoS_UDP
DDoS_ICMP
DDoS_HTTP
DDoS_TCP
这些标签都与分布式拒绝服务（DDoS）攻击有关，只是使用了不同的协议或目标层级。

注入攻击类别：

SQL_injection
XSS
这些标签都与在网站或应用程序中注入恶意代码的攻击有关，只是注入点或技术不同。

身份验证和密码攻击类别：

Password
MITM
这些标签都涉及身份验证和密码方面的攻击，其中"Password"可能指的是针对用户密码的猜测、破解或暴力破解，而"MITM"则是指通过中间人攻击方式获取身份验证凭证。

恶意软件和后门类别：

Ransomware
Backdoor
这些标签都与恶意软件有关，其中"Ransomware"指的是加密文件并勒索赎金，而"Backdoor"指的是创建用于绕过身份验证的隐藏访问途径。

安全扫描类别：

Vulnerability_scanner
Port_Scanning
Fingerprinting
这些标签都与安全扫描和漏洞识别有关，其中"Vulnerability_scanner"指的是用于扫描系统漏洞的工具，而"Port_Scanning"和"Fingerprinting"则是用于识别目标系统的开放端口、配置和特征的技术。

In [73]:
# # 进行分类转换
# df['Attack_type'] = df['Attack_type'].replace({
#     'DDoS_UDP': 'DDoS',
#     'DDoS_ICMP': 'DDoS',
#     'DDoS_HTTP': 'DDoS',
#     'DDoS_TCP': 'DDoS',
#     'SQL_injection': 'Injection Attack',
#     'XSS': 'Injection Attack',
#     'Password': 'Authentication and Password Attack',
#     'MITM': 'Authentication and Password Attack',
#     'Ransomware': 'Malware and Backdoor',
#     'Backdoor': 'Malware and Backdoor',
#     'Vulnerability_scanner': 'Security Scanning',
#     'Port_Scanning': 'Security Scanning',
#     'Fingerprinting': 'Security Scanning'
# })


# print(df['Attack_type'].value_counts())

In [74]:
# # 定义目标采样策略
# sampling_strategy = {'Malware and Backdoor': 10000, 'Uploading': 10000}

# # 创建SMOTE对象
# smote = SMOTE(sampling_strategy=sampling_strategy, n_jobs=-1)

# # 进行过采样
# X_resampled, y_resampled = smote.fit_resample(df.drop('Attack_type', axis=1), df['Attack_type'])

In [75]:
# # 将特征和目标变量转换为DataFrame
# X_resampled_df = pd.DataFrame(X_resampled, columns=df.drop('Attack_type', axis=1).columns)
# y_resampled_df = pd.DataFrame(y_resampled, columns=['Attack_type'])

# df = pd.concat([X_resampled_df, y_resampled_df], axis=1)

# print(df.shape)
# df.head()

In [76]:
# df['Attack_type'].value_counts()

In [77]:
df.to_csv('data/DLL_pic_data.csv',index=0)

### Feature Selection

In [78]:
features = df.dtypes[df.dtypes != 'object'].index
features

Index(['icmp.checksum', 'icmp.seq_le', 'http.content_length',
       'http.request.method', 'http.referer', 'http.request.version',
       'http.response', 'tcp.ack', 'tcp.ack_raw', 'tcp.checksum',
       'tcp.connection.fin', 'tcp.connection.rst', 'tcp.connection.syn',
       'tcp.connection.synack', 'tcp.flags', 'tcp.flags.ack', 'tcp.len',
       'tcp.seq', 'udp.stream', 'udp.time_delta', 'dns.qry.name',
       'dns.qry.name.len', 'dns.qry.qu', 'dns.retransmission',
       'mqtt.conack.flags', 'mqtt.conflag.cleansess', 'mqtt.conflags',
       'mqtt.hdrflags', 'mqtt.len', 'mqtt.msgtype', 'mqtt.proto_len',
       'mqtt.protoname', 'mqtt.topic', 'mqtt.topic_len', 'mqtt.ver',
       'mbtcp.len', 'mbtcp.trans_id', 'mbtcp.unit_id'],
      dtype='object')

In [79]:
# Boruta

from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier

def boruta(X, y):
    rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
    boruta = BorutaPy(rf, n_estimators='auto', verbose=2)
    boruta.fit(X, y)
    return boruta.transform(X)


In [80]:
data_boruta = boruta(np.array(df[features]), list(df['Attack_type']))
joblib.dump(data_boruta, 'data/data_boruta')

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	38
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	38
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	38
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	38
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	38
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	38
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	38
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	35
Tentative: 	0
Rejected: 	3


BorutaPy finished running.

Iteration: 	9 / 100
Confirmed: 	35
Tentative: 	0
Rejected: 	3


['data/data_boruta']

In [46]:
# Relief

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

def relief(X, y, k=None, start=5):
    
    if k== None:
        # 构建一个特征选择器，并使用交叉验证来选择最佳特征数量
        k_range = range(start, X.shape[1])
        cv_scores = []

        for k in k_range:

            fs = SelectKBest(score_func=mutual_info_classif, k=k)
            X_selected = fs.fit_transform(X, y)

            clf = LogisticRegression()
            scores = cross_val_score(clf, X_selected, y, cv=5, scoring='roc_auc')
            cv_scores.append(np.mean(scores))

        # 找到最佳的特征数量 k
        best_k = k_range[np.argmax(cv_scores)]
        print(f"Best number of features: {best_k}")

        # 使用最佳的特征数量 k 进行特征选择
        fs = SelectKBest(score_func=mutual_info_classif, k=best_k)
        
        return fs.fit_transform(X, y)
        
    else:
        fs = SelectKBest(score_func=mutual_info_classif, k=k)
        fs.fit(X, y)
        
        return fs.transform(X)


In [47]:
data_relief = relief(np.array(df[features]), list(df['Attack_type']))

Best number of features: 5


In [48]:
joblib.dump(data_relief, 'data/data_relief')

['data/data_relief']

In [58]:
# Mutual Information Feature Selection（MIFS）

from minepy import MINE
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression


def mic(x, y):
    m = MINE()
    m.compute_score(x, y)
    return m.mic()


def mifs(X, y, k=None, start=5):
    
    if k== None:
        # 构建一个特征选择器，并使用交叉验证来选择最佳特征数量
        k_range = range(start, X.shape[1])
        cv_scores = []

        for k in k_range:

            F = []
            for i in range(X.shape[1]):
                f = 0.0
                for j in range(X.shape[1]):
                    if i != j:
                        f += mic(X[:, i], X[:, j])
                F.append(f)

            fs = SelectKBest(score_func=mutual_info_classif, k=k)
            X_selected = fs.fit_transform(X, y)
            F_selected = np.array(F)[fs.get_support()]

            X_combined = np.hstack((X_selected, F_selected.reshape(-1, 1)))

            clf = LogisticRegression()
            scores = cross_val_score(clf, X_combined, y, cv=5, scoring='roc_auc')
            cv_scores.append(np.mean(scores))

        # 找到最佳的特征数量 k
        best_k = k_range[np.argmax(cv_scores)]
        print(f"Best number of features: {best_k}")

        # 使用最佳的特征数量 k 进行特征选择
        F = []
        for i in range(X.shape[1]):
            f = 0.0
            for j in range(X.shape[1]):
                if i != j:
                    f += mic(X[:, i], X[:, j])
            F.append(f)

        fs = SelectKBest(score_func=mutual_info_classif, k=best_k)
        X_selected = fs.fit_transform(X, y)
        F_selected = np.array(F)[fs.get_support()]

        return np.hstack((X_selected, F_selected.reshape(-1, 1)))
        
    else:
        F = []
        for i in range(X.shape[1]):
            f = 0.0
            for j in range(X.shape[1]):
                if i != j:
                    f += mic(X[:, i], X[:, j])
            F.append(f)

        fs = SelectKBest(score_func=mutual_info_classif, k=k)
        X_selected = fs.fit_transform(X, y)
        F_selected = np.array(F)[fs.get_support()]

        return np.hstack((X_selected, F_selected.reshape(-1, 1)))


In [ ]:
data_mifs = mifs(np.array(df[features]), list(df['Attack_type']))
joblib.dump(data_mifs, 'data/data_mifs')

In [56]:
# Recursive Feature Elimination（RFE）

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

def rfe(X, y, k=None, start=5):
    if k == None:
        # 构建一个特征选择器，并使用交叉验证来选择最佳特征数量
        k_range = range(start, X.shape[1])
        cv_scores = []

        for k in k_range:
            model = LogisticRegression()
            rfe = RFE(estimator=model, n_features_to_select=k)
            X_selected = rfe.fit_transform(X, y)

            scores = cross_val_score(model, X_selected, y, cv=5, scoring='roc_auc')
            cv_scores.append(np.mean(scores))

        # 找到最佳的特征数量 k
        best_k = k_range[np.argmax(cv_scores)]
        print(f"Best number of features: {best_k}")

        # 使用最佳的特征数量 k 进行特征选择
        model = LogisticRegression()
        fs = RFE(model, n_features_to_select=best_k)

        return fs.fit_transform(X, y)
    else:
        model = LogisticRegression()
        fs = RFE(model, n_features_to_select=k)
        fs.fit(X, y)

        return fs.transform(X)


In [57]:
data_rfe = rfe(np.array(df[features]), list(df['Attack_type']))
joblib.dump(data_rfe, 'data/data_rfe')

Best number of features: 5


['data/data_rfe']

In [ ]:
# df_select = pd.DataFrame(data_boruta)
# df_select['Attack_type'] = list(df['Attack_type'])

# df_select.head()

In [ ]:
df_select.describe()

### split train set and test set

In [ ]:
# X = df.drop(['Attack_type'],axis=1).values
# y = df.iloc[:, -1].values.reshape(-1,1)
# y = np.ravel(y)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, 
#                                                     train_size=0.8, test_size=0.2, 
#                                                     random_state=2023, stratify=y
#                                                    )